In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
inputData = pd.read_csv('census_data.csv')

In [3]:
inputData.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
inputData.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [5]:
norm_list = ['age','education_num','capital_gain','capital_loss','hours_per_week']

In [6]:
inputData[norm_list]=inputData[norm_list].apply(lambda x:(x-x.min())/(x.max()-x.min()))

In [7]:
inputData['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [8]:
def change_ib(value):
    if value ==' <=50K':
        return 0
    else:
        return 1

In [9]:
inputData['income_bracket'] = inputData['income_bracket'].apply(change_ib)

In [10]:
inputData.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,0.301370,State-gov,Bachelors,0.800000,Never-married,Adm-clerical,Not-in-family,White,Male,0.02174,0.0,0.397959,United-States,0
1,0.452055,Self-emp-not-inc,Bachelors,0.800000,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.00000,0.0,0.122449,United-States,0
2,0.287671,Private,HS-grad,0.533333,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.00000,0.0,0.397959,United-States,0
3,0.493151,Private,11th,0.400000,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.00000,0.0,0.397959,United-States,0
4,0.150685,Private,Bachelors,0.800000,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.00000,0.0,0.397959,Cuba,0


Create the Feature Columns for tf.esitmator

In [11]:
inputData.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [12]:
gender = tf.feature_column.categorical_column_with_vocabulary_list('gender',['Female','Male'])

In [13]:
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass",
hash_bucket_size = 1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education",
hash_bucket_size = 1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status",
hash_bucket_size = 1000)
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation",
hash_bucket_size = 1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship",
hash_bucket_size = 1000)
country = tf.feature_column.categorical_column_with_hash_bucket("native_country",
hash_bucket_size = 1000)
race = tf.feature_column.categorical_column_with_hash_bucket("race",
hash_bucket_size = 1000)

In [14]:
age  = tf.feature_column.numeric_column("age")
education_num  = tf.feature_column.numeric_column("education_num")
capital_gain  = tf.feature_column.numeric_column("capital_gain")
capital_loss  = tf.feature_column.numeric_column("capital_loss")
hours_per_week  = tf.feature_column.numeric_column("hours_per_week")

In [15]:
feat_cols = [gender,workclass,education,marital_status,occupation,relationship,country,race,age,education_num,capital_gain,capital_loss,hours_per_week]

Perform a Train Test Split on the Data

In [16]:
x_data = inputData.drop('income_bracket',axis = 1)
x_data.shape

(32561, 13)

In [17]:
y_value = inputData['income_bracket']

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_value, test_size=0.3,random_state=101)
X_train.shape, X_test.shape

((22792, 13), (9769, 13))

Create Input Function

In [20]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size = 10,num_epochs = 1000,shuffle = True)

In [21]:
model = tf.estimator.LinearClassifier(feature_columns = feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\User\\AppData\\Local\\Temp\\tmp2o5v39rv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000DA7DC78E48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
model.train(input_fn = input_func,steps = 5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\User\AppData\Local\Temp\tmp2o5v39rv\model.ckpt.
INFO:tensorflow:loss = 6.931472, step = 1
INFO:tensorflow:global_step/sec: 58.4324
INFO:tensorflow:loss = 2.605875, step = 101 (1.727 sec)
INFO:tensorflow:global_step/sec: 235.854
INFO:tensorflow:loss = 3.9471009, step = 201 (0.408 sec)
INFO:tensorflow:global_step/sec: 217.971
INFO:tensorflow:loss = 4.3171177, step = 301 (0.459 sec)
INFO:tensorflow:global_step/sec: 32.1976
INFO:tensorflow:loss = 3.044972, step = 401 (3.108 sec)
INFO:tensorflow:global_step/sec: 225.304
INFO:tensorflow:loss = 2.3000863, step = 501 (0.442 sec)
INFO:tensorflow:global_step/sec: 204.392
INFO:tensorflow:loss = 3.8803902, step = 601 (0.489 sec)
INFO:tensorflow:global_step/sec:

In [23]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      y=y_test,
      batch_size=100,
      num_epochs=1,
      shuffle=False)

In [24]:
predictions = model.predict(eval_input_func)

In [25]:
pred = list(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\User\AppData\Local\Temp\tmp2o5v39rv\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [26]:
for prediction in pred:
    print(prediction)

{'logits': array([-0.7966095], dtype=float32), 'logistic': array([0.31075123], dtype=float32), 'probabilities': array([0.68924874, 0.31075123], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-5.5107546], dtype=float32), 'logistic': array([0.00402677], dtype=float32), 'probabilities': array([0.99597317, 0.00402677], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.08938134], dtype=float32), 'logistic': array([0.47766954], dtype=float32), 'probabilities': array([0.52233046, 0.47766954], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-6.7411923], dtype=float32), 'logistic': array([0.00117984], dtype=float32), 'probabilities': array([0.9988201 , 0.00117984], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.4203987], dtype=float32), 

{'logits': array([-4.9272866], dtype=float32), 'logistic': array([0.00719401], dtype=float32), 'probabilities': array([0.992806  , 0.00719401], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.9155486], dtype=float32), 'logistic': array([0.12835878], dtype=float32), 'probabilities': array([0.8716412 , 0.12835878], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-5.294192], dtype=float32), 'logistic': array([0.00499559], dtype=float32), 'probabilities': array([0.9950045 , 0.00499559], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.8110292], dtype=float32), 'logistic': array([0.30767122], dtype=float32), 'probabilities': array([0.6923288 , 0.30767125], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.177366], dtype=float32), 'lo

{'logits': array([1.3253729], dtype=float32), 'logistic': array([0.79007417], dtype=float32), 'probabilities': array([0.20992576, 0.79007417], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-1.1679893], dtype=float32), 'logistic': array([0.23721863], dtype=float32), 'probabilities': array([0.7627814 , 0.23721863], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.7074267], dtype=float32), 'logistic': array([0.33016768], dtype=float32), 'probabilities': array([0.6698323 , 0.33016768], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.4006388], dtype=float32), 'logistic': array([0.03227551], dtype=float32), 'probabilities': array([0.96772456, 0.03227551], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-5.1257977], dtype=float32), 'l

{'logits': array([0.52116984], dtype=float32), 'logistic': array([0.62742126], dtype=float32), 'probabilities': array([0.3725787 , 0.62742126], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-3.1414328], dtype=float32), 'logistic': array([0.04143018], dtype=float32), 'probabilities': array([0.95856977, 0.04143018], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.8098176], dtype=float32), 'logistic': array([0.02167214], dtype=float32), 'probabilities': array([0.9783279 , 0.02167214], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-4.6606193], dtype=float32), 'logistic': array([0.00937194], dtype=float32), 'probabilities': array([0.9906281 , 0.00937194], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.07472777], dtype=float32), 

{'logits': array([-2.333055], dtype=float32), 'logistic': array([0.0884221], dtype=float32), 'probabilities': array([0.91157794, 0.08842211], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.189338], dtype=float32), 'logistic': array([0.03956893], dtype=float32), 'probabilities': array([0.9604311 , 0.03956893], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.8424807], dtype=float32), 'logistic': array([0.30101258], dtype=float32), 'probabilities': array([0.6989874 , 0.30101258], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.219819], dtype=float32), 'logistic': array([0.03842667], dtype=float32), 'probabilities': array([0.9615733 , 0.03842667], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.2289767], dtype=float32), 'logi

...

{'logits': array([-2.938587], dtype=float32), 'logistic': array([0.0502787], dtype=float32), 'probabilities': array([0.9497213, 0.0502787], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([1.7541544], dtype=float32), 'logistic': array([0.85247606], dtype=float32), 'probabilities': array([0.14752397, 0.85247606], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-3.2683861], dtype=float32), 'logistic': array([0.0366718], dtype=float32), 'probabilities': array([0.9633282, 0.0366718], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.04285866], dtype=float32), 'logistic': array([0.489287], dtype=float32), 'probabilities': array([0.51071304, 0.489287  ], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([1.5185242], dtype=float32), 'logistic':

{'logits': array([-1.8880556], dtype=float32), 'logistic': array([0.13146633], dtype=float32), 'probabilities': array([0.8685337 , 0.13146633], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.07294655], dtype=float32), 'logistic': array([0.48177144], dtype=float32), 'probabilities': array([0.51822853, 0.48177144], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([0.9811531], dtype=float32), 'logistic': array([0.72733694], dtype=float32), 'probabilities': array([0.27266306, 0.72733694], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-4.360323], dtype=float32), 'logistic': array([0.01261314], dtype=float32), 'probabilities': array([0.9873869 , 0.01261314], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.9559112], dtype=float32), 'l

{'logits': array([-5.6420474], dtype=float32), 'logistic': array([0.00353307], dtype=float32), 'probabilities': array([0.9964669 , 0.00353307], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-6.806591], dtype=float32), 'logistic': array([0.00110524], dtype=float32), 'probabilities': array([0.99889475, 0.00110524], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.1593559], dtype=float32), 'logistic': array([0.10346019], dtype=float32), 'probabilities': array([0.89653975, 0.10346017], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.009291], dtype=float32), 'logistic': array([0.11823089], dtype=float32), 'probabilities': array([0.8817691 , 0.11823088], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-5.2314186], dtype=float32), 'lo

{'logits': array([-6.5144925], dtype=float32), 'logistic': array([0.00147962], dtype=float32), 'probabilities': array([0.99852043, 0.00147962], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-5.78052], dtype=float32), 'logistic': array([0.00307761], dtype=float32), 'probabilities': array([0.9969223 , 0.00307761], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.5792398], dtype=float32), 'logistic': array([0.17090316], dtype=float32), 'probabilities': array([0.82909685, 0.17090316], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.2625318], dtype=float32), 'logistic': array([0.2205384], dtype=float32), 'probabilities': array([0.7794616 , 0.22053836], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.45939457], dtype=float32), 'lo

{'logits': array([-2.708762], dtype=float32), 'logistic': array([0.06245831], dtype=float32), 'probabilities': array([0.93754166, 0.06245831], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([4.2107997], dtype=float32), 'logistic': array([0.9853823], dtype=float32), 'probabilities': array([0.01461765, 0.9853823 ], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([0.13129443], dtype=float32), 'logistic': array([0.53277653], dtype=float32), 'probabilities': array([0.46722347, 0.53277653], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-5.036402], dtype=float32), 'logistic': array([0.00645514], dtype=float32), 'probabilities': array([0.99354494, 0.00645514], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.03373533], dtype=float32), 'log

In [27]:
final_preds = [prediction['class_ids'][0] for prediction in pred]
final_preds

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


In [28]:
from sklearn.metrics import classification_report

In [29]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90      7436
           1       0.70      0.61      0.65      2333

   micro avg       0.84      0.84      0.84      9769
   macro avg       0.79      0.76      0.78      9769
weighted avg       0.84      0.84      0.84      9769

